# Exploratory Data Analysis of Safety Dataset

## Executive Summary

## Introduction

## About the Data

## Preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# check the dataset folders
!ls data/safety/

data_dictionary.xlsx  features	labels


In [3]:
# read the data dictionary
xl = pd.ExcelFile("data/safety/data_dictionary.xlsx")

In [4]:
# sheet names
xl.sheet_names

['telematics data', 'labels']

In [5]:
# let's see the first sheet
xl.parse('telematics data').head()

,Unnamed: 0,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,description,trip id,accuracy inferred by GPS in meters,GPS bearing,accelerometer reading in x axis (m/s2),accelerometer reading in y axis (m/s2),accelerometer reading in z axis (m/s2),gyroscope reading in x axis (rad/s),gyroscope reading in y axis (rad/s),gyroscope reading in z axis (rad/s),time of the record by number of seconds,speed measured by GPS in m/s
1,samples,1,5,303.695,-0.00636292,-0.393829,-0.922379,"-0.020000606102604086,0.03205247529964867,-0.0...",NaN,NaN,0,0.57
2,NaN,1,10,325.39,0.183914,-0.355026,-0.92041,"-0.028598887998033916,0.025720543491876274,-0....",NaN,NaN,1,0.28
3,NaN,1,5,303.695,-0.00636292,-0.392944,-0.922226,"-0.01894040167264354,0.030980020328673762,-0.0...",NaN,NaN,2,0.57
4,NaN,1,10,324.23,0.165924,-0.332092,-0.920578,"-0.0577245492596855,0.002558232543130116,0.014...",NaN,NaN,3,0.28


In [6]:
# let's see the second sheet
xl.parse('labels').head()

,bookingID,label
0,1,1
1,2,0
2,3,0
3,4,0
4,5,1


In [7]:
# let's see the contents of features and labels folder
!ls data/safety/features

# there are many csvs, we either want to run this in Pyspark 
# or we can combine them into just one Pandas dataset

part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv


In [8]:
!ls data/safety/labels

part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv


In [9]:
# let's load one features csv, there are many csvs we either
_ = pd.read_csv("data/safety/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
_.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000
1,274877907034,9.293,17.0,0.032775,8.659933,4.737300,0.024629,0.004028,-0.010858,257.0,0.190000
2,884763263056,3.000,189.0,1.139675,9.545974,1.951334,-0.006899,-0.015080,0.001122,973.0,0.667059
3,1073741824054,3.900,126.0,3.871543,10.386364,-0.136474,0.001344,-0.339601,-0.017956,902.0,7.913285
4,1056561954943,3.900,50.0,-0.112882,10.550960,-1.560110,0.130568,-0.061697,0.161530,820.0,20.419409


In [10]:
# let's load the labels csv
labels = pd.read_csv("data/safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")
labels.head()

,bookingID,label
0,111669149733,0
1,335007449205,1
2,171798691856,0
3,1520418422900,0
4,798863917116,0


In [11]:
# how many labels are there?
len(labels)

20018

In [12]:
# how many unique bookings are there?
len(labels.bookingID.unique())

20000

### Loading using Spark

In [13]:
# import pyspark for parallel processing of data
# Import SparkContext from pyspark - used to connect to cluster
from pyspark import SparkContext

# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

In [14]:
# Create sparksession
# Create spark
spark = SparkSession.builder.getOrCreate()

In [15]:
# creating a spark dataframe
feats = spark.read.csv("data/safety/features/*.csv", header=True, inferSchema=True)

# create table named feats
feats.createOrReplaceTempView("feats")

type(feats)

pyspark.sql.dataframe.DataFrame

In [16]:
# showing the first 5 items
feats.show(5)

+-------------+--------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+--------------------+------+------------------+
|    bookingID|Accuracy|           Bearing|   acceleration_x|    acceleration_y|     acceleration_z|              gyro_x|              gyro_y|              gyro_z|second|             Speed|
+-------------+--------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+--------------------+------+------------------+
|1657857376413|    10.0|128.23483276367188|0.780279541015625|-9.869085693359375| -0.019439697265625|-0.14392361689449823|-0.01666206736008...|-0.01698644038085...| 497.0|16.030000686645508|
|  25769803913|     6.0|286.36822509765625|9.086712646484376|0.3093902587890625|-3.5067718505859378|0.008517588114400428|-0.00635030761929...|0.003299390274499...| 120.0| 2.089625835418701|
|1434519076946|  12.136|             339.0|       

In [17]:
# showing the columns
feats.columns

['bookingID',
 'Accuracy',
 'Bearing',
 'acceleration_x',
 'acceleration_y',
 'acceleration_z',
 'gyro_x',
 'gyro_y',
 'gyro_z',
 'second',
 'Speed']

In [18]:
# counting how many rows
query = "SELECT COUNT(*) FROM feats"
spark.sql(query).show()

+--------+
|count(1)|
+--------+
|16135561|
+--------+



In [19]:
# counting the number of unique booking IDs
query = "SELECT COUNT(DISTINCT(bookingID)) FROM feats"
spark.sql(query).show()

+-------------------------+
|count(DISTINCT bookingID)|
+-------------------------+
|                    20000|
+-------------------------+



In [20]:
# load labels
# creating a spark dataframe
labels = spark.read.csv("data/safety/labels/*.csv", header=True, inferSchema=True)

# create table named feats
labels.createOrReplaceTempView("labels")

type(labels)

pyspark.sql.dataframe.DataFrame

In [21]:
# showing the first 5 items
labels.show(5)

+-------------+-----+
|    bookingID|label|
+-------------+-----+
| 111669149733|    0|
| 335007449205|    1|
| 171798691856|    0|
|1520418422900|    0|
| 798863917116|    0|
+-------------+-----+
only showing top 5 rows



In [22]:
# counting how many rows
query = "SELECT COUNT(*) FROM labels"
spark.sql(query).show()

+--------+
|count(1)|
+--------+
|   20018|
+--------+



In [28]:
# counting the number of unique rows
query = """SELECT COUNT(*) FROM (SELECT bookingID, label, COUNT(*) as count FROM labels
            GROUP BY bookingID, label
            HAVING COUNT(*) = 1)"""
spark.sql(query).show()

+--------+
|count(1)|
+--------+
|   20018|
+--------+



In [29]:
# counting the number of unique booking IDs
query = "SELECT COUNT(DISTINCT(bookingID)) FROM labels"
spark.sql(query).show()

+-------------------------+
|count(DISTINCT bookingID)|
+-------------------------+
|                    20000|
+-------------------------+



In [ ]:
# How do we handle the 18 others?

In [24]:
# Merge the features and labels datasets by bookingID
query = "SELECT"

## EDA